*********************************************************************************************************
# A Tour of Python 3  
version 1.0.1  
Authors: Phil Pfeiffer, Zack Bunch, and Feyisayo Oyeniyi  
East Tennessee State University  
Last updated June 2021  
*********************************************************************************************************

# 11.  OO programming in Python   
 11.1 [Encapsulation and data hiding](#OO-Programming-In-Python-Encapsulation-And-Data-Hiding)  
 11.2 [Dependency inversion](#OO-Programming-In-Python-Dependency-Inversion)  
 11.3 [Nested classes](#OO-Programming-In-Python-Nested-Classes)

## 11.1  Encapsulation and data hiding <a name='OO-Programming-In-Python-Encapsulation-And-Data-Hiding'></a>

Like other OO languages, Python supports encapsulation. If an object A contains an object B, then B must be accessed "through" A, using an expression like `A.B`. The one exception to this rule involves `import` statements that expose a package's objects for direct use, like *from A import B*. 

Unlike compiled OO languages, Python lacks support for strong data hiding. Declarations like "public", "private" and "protected" are absent from Python. Rather, **data hiding in Python is a programmer-enforced convention**. Apart from a few built-in immutable classes and constants, all Python identifiers are always readable and updateable, as are the objects they reference. The rationale for this lack of checking is threefold:
-  Python's support for `eval` and `exec` renders any attempt to implement static access checking imperfect.
-  Using run-time checks to enforce data hiding would slow every data access.  Most checks, moreover, would be useless and redundant.
-  Treating data hiding as a convention rather than a requirement gives programmers freedom to break data hiding at need.

Python does support one weak mechanism for data hiding.  This mechanism, *name mangling*, is invoked automatically for attributes whose names start with a double underscore.  Python manages these attributes by
-  assigning them new names and
-  converting all references to these attributes from within their container objects to these new names

While name mangling effectively invalidates all external references to the "real" names, this transformation is easily circumvented.

In [ ]:
# 11.1.a  'weak' data hiding using data mangling

class MyClass:
  def __init__(self, v):  self.__value = v
  def get_value(self):    return self.__value

instance = MyClass(1)
print( 'result of using get_value() to retrieve value from instance: ', instance.get_value() ) 

print( 'result of reading instance.__value: ', end='' )
try:
  print( instance.__value )
except:
  print( 'failure' )

print( 'result of setting instance.__value to 4, then using instance.get_value() to read it: ', end='' )
try:
  instance.__value = 4
  print( instance.get_value() )
except:
  print( 'failure' )

print()
print( 'result of reading instance._MyClass__value: ', end='' )
try:
  print( instance._MyClass__value )
except:
  print( 'failure' )

print( 'result of setting instance._MyClass__value to 4, then using instance.get_value() to read it: ', end='' )
try:
  instance._MyClass__value = 4
  print( instance.get_value() )
except:
  print( 'failure' )

##  11.2 Dependency inversion <a name='OO-Programming-In-Python-Dependency-Inversion'></a>

*Dependency inversion* refers to the structuring of a family of objects as objects with a single, common interface and varied implementations. Type systems for compiled OO languages require that the common interface be defined as a base class, then specialized for particular realizations of that interface. A classic characterization of this approach posits 
-  an abstract base class (ABC) that returns a time of day, with 
-  a set of child classes that retrieve time-of-day information from different kinds of timepieces, 
   ranging from digital assistants to digital clocks to sundials.

The term "dependency inversion" was coined at a time when object-oriented languages first came into vogue. It was developed as an alternative to *run-time type inference* (RTTI):  a design strategy that uses a test of a value's type at run time to select one of collection of routines to use to process that value. The term *inversion* was used to signal a need to "invert" previous practice by designing an ABC before its child classes. 

RTTI is now viewed as a poor practice that hinders program evolution. In RTTI-based codes, adding routines to or removing them from a given family of routines can force changes to *all* codes that access *any* of these routines.  Dependency inversion, by contrast, localizes such changes to the classes being added or removed, so long as those classes share the common interface.

In interpreted languages, the lack of compile-time type checking eliminates the need to use a base class in order to assure a child class's type is correct for where it's being used. So long as an object has attributes that fit a given context, its type is assumed to be correct. This approach to type checking is referred to as *latent* typing or *duck* typing: i.e., if an object walks like a duck and quacks like a duck, assume it's a duck.

Latent typing is favored by a minimalist school of software design, including authorities like Russ Olsen (<u>Eloquent Ruby</u>) who argue for achieving software quality by writing concise programs with clearly named variables and supporting them with carefully crafted regression test suites. These authorities criticize type declarations and hard-coded type checks as syntactic clutter: i.e., attempt to prevent potential errors that would be better prevented by careful naming and testing. While these authorities recommend inheritance as a means of eliminating duplicate code, they see it as useless for defining types.

The following examples use these additional Python constructs:
- `ABCMeta` - a Python *metaclass* (see below) that prevents a class from being used to instantiate other classes; 
  the constrained class is similar to abstract base classes in compiled OO languages.
-  `input` - read a string from the user's terminal
-  `time.strftime` - display time of day as a string

A *metaclass* is a class that controls the operation of a(nother) class.   All Python classes have metaclasses: the default is `object`.   For more on programming with metaclasses and reasons for thinking twice before doing so, see (e.g.) 
- Mark Lutz's <u>Learning Python</u>, 5th edition
- Mark Summerfield's <u>Python3</u>

In [ ]:
# 11.2.a   An RTTI-based code (bad!) that
# -  solicits an "option 1 or 2" answer from a user in one of several languages, then
# -  launches a command in response to this response

# supporting constants
TIME_AS_24_HR_FORMAT = '%H:%M'      # time.strftime string for military time format
TIME_AS_12_HR_FORMAT = '%I:%M %p'   # time.strftime string for am/pm time format
OPTSTRINGS = {'1': TIME_AS_24_HR_FORMAT, '2': TIME_AS_12_HR_FORMAT}

# supporting library modules
import time

def QueryInEnglish():
  while True:
    option = input('Please enter 1 for 24 hour format, 2 for AM/PM format: ')
    if option in OPTSTRINGS.keys(): return option

def QueryInFrench():
  while True:
    option = input('S\'il vous plaît entrer 1 pour le format de 24 heures, 2 pour le format AM / PM: ')
    if option in OPTSTRINGS.keys(): return option

def QueryInSpanish():
  while True:
    option = input('Por favor, introduzca 1 para el formato de 24 horas, 2 para el formato AM / PM: ')
    if option in OPTSTRINGS.keys(): return option

def get_time(language):
  if language=="EN":
    return time.strftime(OPTSTRINGS[QueryInEnglish()])
  elif language=="FR":
    return time.strftime(OPTSTRINGS[QueryInFrench()])
  elif language=="ES":
    return time.strftime(OPTSTRINGS[QueryInSpanish()])
  else:
    return f'language not supported: {language}' 

print( 'Time in English is', get_time("EN"), '\n' )
print( 'Time in French is',  get_time("FR"), '\n' )
print( 'Time in Spanish is', get_time("ES"), '\n' )
print( 'Time in Italian is', get_time("IT") )

In [ ]:
# 11.2.b   redoing the previous example using dependency inversion
#  the following code encapsulates queries as classes with an abstract base class,
#  and features a type check for the get_time() routine parameter
#
#  TwoOptionQuery metaclasses are classes that control the instantiation of classes
#  for more on metaclasses, see the resources named at the outset of this document

# supporting constants
TIME_AS_24_HR_FORMAT = '%H:%M'      # time.strftime string for military time format
TIME_AS_12_HR_FORMAT = '%I:%M %p'   # time.strftime string for am/pm time format
OPTSTRINGS = {'1': TIME_AS_24_HR_FORMAT, '2': TIME_AS_12_HR_FORMAT}

# supporting library modules
import time
from abc import ABCMeta

# supporting functions
make_printable = lambda exception: '' if str(exception) is None else str(exception)

class TwoOptionQuery(metaclass=ABCMeta):
  @classmethod
  def getOption(clas):
    while True:
      option = input(clas.querystring)
      if option in OPTSTRINGS.keys(): return option

class QueryInEnglish(TwoOptionQuery):
  querystring = 'Please enter 1 for 24 hour format, 2 for AM/PM format: '

class QueryInFrench(TwoOptionQuery):
  querystring = 'S\'il vous plaît entrer 1 pour le format de 24 heures, 2 pour le format AM / PM: '

class QueryInSpanish(TwoOptionQuery):
  querystring = 'Por favor, introduzca 1 para el formato de 24 horas, 2 para el formato AM / PM: '

class QueryInItalian:
  @staticmethod
  def getOption():
    while True:
      option = input( 'Inserisci 1 per il formato 24 ore, 2 per il formato AM / PM: ' )
      if option in OPTSTRINGS.keys(): return option

def get_time(querier):
  # the following check would be automatic in compiled OO languages 
  # in Python, it's optional - and included for the sake of completeness
  assert issubclass(querier, TwoOptionQuery), f'querier ({querier.__name__}) must be a subclass of TwoOptionQuery' 
  return time.strftime({'1': TIME_AS_24_HR_FORMAT, '2': TIME_AS_12_HR_FORMAT}[querier.getOption()])

print( 'Time in English is', get_time(QueryInEnglish), '\n' )
print( 'Time in French is',  get_time(QueryInFrench), '\n' )
print( 'Time in Spanish is', get_time(QueryInSpanish), '\n' )
try:
  print( 'Time in Italian is' )
  print( get_time(QueryInItalian) )
except Exception as exception:
  print( '??',make_printable(exception) )

print( '\n\nHowever, the check in get_time can be bypassed by invoking the subclass directly\n' )
time.strftime({'1': "%H:%M", '2': "%I:%M %p"}[QueryInItalian.getOption()])

In [ ]:
# 11.2.c  simplifying the previous example, per "duck typing" practice.
#  -.  the TwoOptionQuery superclass is retained, because it provides common implementation logic
#      for its child classes.
#  -.  the ABCMeta designation has been discarded, however, as the assertion in get_time

# supporting constants
TIME_AS_24_HR_FORMAT = '%H:%M'      # time.strftime string for military time format
TIME_AS_12_HR_FORMAT = '%I:%M %p'   # time.strftime string for am/pm time format
OPTSTRINGS = {'1': TIME_AS_24_HR_FORMAT, '2': TIME_AS_12_HR_FORMAT}

# supporting library modules
import time

class TwoOptionQuery:
  @classmethod
  def getOption(clas):
    while True:
      option = input(clas.querystring)
      if option in OPTSTRINGS.keys(): return option

class QueryInEnglish(TwoOptionQuery):
  querystring = 'Please enter 1 for 24 hour format, 2 for AM/PM format: '

class QueryInFrench(TwoOptionQuery):
  querystring = 'S\'il vous plaît entrer 1 pour le format de 24 heures, 2 pour le format AM / PM: '

class QueryInSpanish(TwoOptionQuery):
  querystring = 'Por favor, introduzca 1 para el formato de 24 horas, 2 para el formato AM / PM: '

class QueryInItalian:
  @staticmethod
  def getOption():
    while True:
      option = input('Inserisci 1 per il formato 24 ore, 2 per il formato AM / PM: ')
      if option in OPTSTRINGS.keys(): return option

def get_time(querier):
  return time.strftime(OPTSTRINGS[querier.getOption()])


print( 'Time in English is', get_time(QueryInEnglish) )
print( 'Time in French is',  get_time(QueryInFrench) )
print( 'Time in Spanish is', get_time(QueryInSpanish) )
print( 'Time in Italian is', get_time(QueryInItalian) )

In addition to `ABCMeta`, the Python library provides decorators that treat a class's methods and properties as abstract methods and properties: i.e., objects that the class's subclasses are required to overload.  See the Python Library documentation on [abstract base classes for containers](https://docs.python.org/3/library/collections.abc.html) for details.

##  11.3 Nested classes  <a name='OO-Programming-In-Python-Nested-Classes'></a>

Python supports the nesting of classes within classes. While nesting tends to interfere with readability, it can be used to express one class's dependency on a second class.

In [ ]:
# 11.3   A representation of a common-- though far from the only-- ranking of cards in card decks.
# For simplicity, the design assumes that the ranking is uniform by suit - 
# an assumption that fails to hold for (e.g.) euchre-family card games.

class CardValues(object):
  __defaultValues = ['ace', 'king', 'queen', 'jack', '10', '9', '8', '7', '6', '5', '4', '3', '2']
  def __init__(self, **kwargs):
    self.card_values = kwargs.get( 'values', CardValues.__defaultValues )
  def __eq__(self, other):
    return isinstance(other, CardValues) and len(self) == len(other) and all([s==o for (s,o) in zip(self,other)])
  @staticmethod
  def outranks(card1, card2):
    assert card2.value in card1.deck.card_values, f'value ({card2.value}) not in values ({card1.deck.card_values})' 
    return card1.deck.card_values.index(card1.value) < card1.deck.card_values.index(card2.value)

class CardSuits(object):
  __defaultSuits = ['spades', 'hearts', 'diamonds', 'clubs']
  def __init__(self, **kwargs):
    self.card_suits = kwargs.get( 'suits', CardSuits.__defaultSuits )
  def __eq__(self, other):
    return isinstance(other, CardSuits) and len(self) == len(other) and all([s==o for (s,o) in zip(self,other)])
  @staticmethod
  def outranks(card1, card2):
    assert card2.suit in card1.deck.card_suits, f'suit ({card2.suit}) not in suits ({card1.deck.card_suits})' 
    return card1.deck.card_suits.index(card1.suit) < card1.deck.card_suits.index(card2.suit)

class CardDeck(CardValues, CardSuits):
  class Card(object):
    def __init__(self, deck, suit, value):
      self.deck, self.suit, self.value = deck, suit, value
    def comparable(self, other):
      return isinstance(other, CardDeck.Card) \
        and self.deck.card_suits == other.deck.card_suits and self.deck.card_values == other.deck.card_values
    def __eq__(self, other):
      return self.comparable(other) and self.suit == other.suit and self.value == other.value
    def __gt__(self, other):
      return self.comparable(other) and deck.outranks(self,other)
    def __lt__(self, other):
      raise NotImplemented
  def __init__(self, **kwargs):
    CardSuits.__init__(self, **kwargs)
    CardValues.__init__(self, **kwargs)
  def getCard(self, suit, value):
    assert suit in self.card_suits,    f"suit  ({suit}) missing from deck's suits  ({self.card_suits})" 
    assert value in self.card_values,  f"value ({value}) missing from deck's values ({self.card_values})" 
    return self.Card(self, suit, value)       # can also be CardDeck.Card
  def __eq__(self, other):
    return isinstance(other, CardDeck) \
      and self.card_suits == other.card_suits and self.card_values == other.card_values
  @staticmethod
  def outranks(card1, card2):
    assert isinstance(card1, CardDeck.Card),  f'object ({card1}) not a card in a card deck' 
    assert isinstance(card2, CardDeck.Card),  f'object ({card2}) not a card in a card deck' 
    if card1.deck != card2.deck:  return False
    if card1.suit != card2.suit:  return False
    return CardValues.outranks(card1,card2)

# some operations on cards...

def compare_cards( c1, c2 ):
  if c1 > c2: return 'outranks'
  if c2 > c1: return 'is outranked by'
  if c1 == c2: return 'equals'
  return 'is incomparable to'

deck = CardDeck()
ace_spades = deck.getCard('spades', 'ace')
deuce_spades = deck.getCard('spades', '2')
ace_clubs = deck.getCard('clubs', 'ace')
deuce_clubs = deck.getCard('clubs', '2')

# comparisons within suits
print( f'The 2 of clubs {compare_cards( deuce_clubs, deuce_clubs )} the 2 of clubs' )
print( f'The 2 of clubs {compare_cards( deuce_clubs, ace_clubs )} the ace of clubs' )
print( f'The ace of clubs {compare_cards( ace_clubs, deuce_clubs )} the 2 of clubs' )
print( f'The ace of clubs {compare_cards( ace_clubs, ace_clubs )} the ace of clubs' )
print( f'The 2 of spades {compare_cards( deuce_spades, deuce_spades )} the 2 of spades' )
print( f'The 2 of spades {compare_cards( deuce_spades, ace_spades )} the ace of spades' )
print( f'The ace of spades {compare_cards( ace_spades, deuce_spades )} the 2 of spades' )
print( f'The ace of spades {compare_cards( ace_spades, ace_spades )} the ace of spades' )

# comparisons across suits
print( f'The 2 of clubs {compare_cards( deuce_clubs, deuce_spades )} the 2 of spades' )
print( f'The 2 of clubs {compare_cards( deuce_clubs, ace_spades )} the ace of spades' )
print( f'The 2 of spades {compare_cards( deuce_spades, deuce_clubs )} the 2 of clubs' )
print( f'The 2 of spades {compare_cards( deuce_spades, ace_clubs )} the ace of clubs' )
print( f'The ace of clubs {compare_cards( ace_clubs, deuce_spades )} the 2 of spades' )
print( f'The ace of clubs {compare_cards( ace_clubs, ace_spades )} the ace of spades' )
print( f'The ace of spades {compare_cards( ace_spades, deuce_clubs )} the 2 of clubs' )
print( f'The ace of spades {compare_cards( ace_spades, ace_clubs )} the ace of clubs' )

<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 11.3.1:**

</span><span style='color:navy'>In the following code cell, modify the previous example so that ranking is determined by card within suit; i.e., so that all spades outrank all clubs. Provide examples that show your change was correct.</span>